In [52]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import tensorflow as tf
import matplotlib.pylab as plt
import os

In [53]:
train = pd.read_csv('train.csv', index_col='id')# /content/drive/My Drive/Colab Notebooks/DACON/data/
test = pd.read_csv('test.csv', index_col='id')
sample_submission = pd.read_csv('sample_submission.csv', index_col='id')

train.head()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
id,,,,,,,,,,,,,,,,,,,,,
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,...,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,NaN
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,...,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,NaN
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,...,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,NaN
3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,...,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,NaN
4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,...,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,NaN


In [54]:
train.tail()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
id,,,,,,,,,,,,,,,,,,,,,
4747,19.9,987.6,0.9,0.8,0.0,1006.9,987.7,21.7,1007.5,1007.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4748,19.9,987.6,0.5,0.7,0.0,1006.8,987.7,21.6,1007.5,1007.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4749,19.7,987.7,0.9,0.6,0.0,1006.9,987.6,21.4,1007.4,1007.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4750,19.4,987.7,0.9,0.8,0.0,1006.9,987.8,21.3,1007.6,1007.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4751,19.1,987.6,1.0,0.3,0.0,1006.8,987.8,21.2,1007.5,1007.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0


In [55]:
train.info() # 결측치 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4752 entries, 0 to 4751
Data columns (total 59 columns):
X00    4752 non-null float64
X01    4752 non-null float64
X02    4752 non-null float64
X03    4752 non-null float64
X04    4752 non-null float64
X05    4752 non-null float64
X06    4752 non-null float64
X07    4752 non-null float64
X08    4752 non-null float64
X09    4752 non-null float64
X10    4752 non-null float64
X11    4752 non-null float64
X12    4752 non-null float64
X13    4752 non-null float64
X14    4752 non-null float64
X15    4752 non-null float64
X16    4752 non-null float64
X17    4752 non-null float64
X18    4752 non-null float64
X19    4752 non-null float64
X20    4752 non-null float64
X21    4752 non-null float64
X22    4752 non-null float64
X23    4752 non-null float64
X24    4752 non-null float64
X25    4752 non-null float64
X26    4752 non-null float64
X27    4752 non-null float64
X28    4752 non-null float64
X29    4752 non-null float64
X30    4752 non-null fl

In [56]:
test.info() # 결측치 없음 # train보다 두 배 이상 데이터가 많음

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11520 entries, 4752 to 16271
Data columns (total 40 columns):
X00    11520 non-null float64
X01    11520 non-null float64
X02    11520 non-null float64
X03    11520 non-null float64
X04    11520 non-null float64
X05    11520 non-null float64
X06    11520 non-null float64
X07    11520 non-null float64
X08    11520 non-null float64
X09    11520 non-null float64
X10    11520 non-null float64
X11    11520 non-null float64
X12    11520 non-null float64
X13    11520 non-null float64
X14    11520 non-null float64
X15    11520 non-null float64
X16    11520 non-null float64
X17    11520 non-null float64
X18    11520 non-null float64
X19    11520 non-null float64
X20    11520 non-null float64
X21    11520 non-null float64
X22    11520 non-null float64
X23    11520 non-null float64
X24    11520 non-null float64
X25    11520 non-null float64
X26    11520 non-null float64
X27    11520 non-null float64
X28    11520 non-null float64
X29    11520 non-n

In [57]:
Y_train = train.loc[:4319, 'Y00':'Y17']
Y_train.head()

,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17
id,,,,,,,,,,,,,,,,,,
0,12.5,11.5,11.0,20.0,20.0,10.5,10.0,10.5,10.0,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0
1,12.5,11.0,11.0,20.0,20.0,10.0,9.5,10.0,9.5,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0
2,12.0,11.0,11.0,19.5,19.5,10.0,9.0,10.0,9.0,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5
3,12.0,11.0,11.0,19.5,19.5,9.5,9.0,9.5,9.0,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5
4,12.0,10.5,10.5,19.0,19.5,9.5,8.5,9.5,8.5,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5


In [58]:
Y_train.tail()

,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17
id,,,,,,,,,,,,,,,,,,
4315,22.0,22.5,22.5,26.0,26.0,21.0,20.0,20.5,20.5,19.5,20.0,19.0,20.5,20.5,20.0,20.0,20.5,20.0
4316,22.0,22.5,22.5,26.0,25.5,21.0,20.0,20.5,20.5,19.5,20.0,19.0,20.5,20.5,20.0,19.5,20.5,19.5
4317,21.5,22.5,22.5,26.0,25.5,21.0,20.0,20.5,20.5,19.5,20.0,19.0,20.0,20.0,19.5,19.5,20.0,19.0
4318,21.5,22.0,22.0,25.5,25.5,20.5,20.0,20.0,20.0,19.5,20.0,18.5,20.0,20.0,19.5,19.5,20.0,19.0
4319,21.5,22.0,22.0,25.5,25.5,20.5,19.5,20.0,20.0,19.5,20.0,18.5,19.5,19.5,19.5,19.5,19.5,19.0


In [59]:
Y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4320 entries, 0 to 4319
Data columns (total 18 columns):
Y00    4320 non-null float64
Y01    4320 non-null float64
Y02    4320 non-null float64
Y03    4320 non-null float64
Y04    4320 non-null float64
Y05    4320 non-null float64
Y06    4320 non-null float64
Y07    4320 non-null float64
Y08    4320 non-null float64
Y09    4320 non-null float64
Y10    4320 non-null float64
Y11    4320 non-null float64
Y12    4320 non-null float64
Y13    4320 non-null float64
Y14    4320 non-null float64
Y15    4320 non-null float64
Y16    4320 non-null float64
Y17    4320 non-null float64
dtypes: float64(18)
memory usage: 641.2 KB


In [60]:
Y_train['Y18']=Y_train.mean(axis=1)
Y_train.head()

,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
id,,,,,,,,,,,,,,,,,,,
0,12.5,11.5,11.0,20.0,20.0,10.5,10.0,10.5,10.0,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,10.666667
1,12.5,11.0,11.0,20.0,20.0,10.0,9.5,10.0,9.5,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,10.444444
2,12.0,11.0,11.0,19.5,19.5,10.0,9.0,10.0,9.0,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,10.166667
3,12.0,11.0,11.0,19.5,19.5,9.5,9.0,9.5,9.0,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,10.000000
4,12.0,10.5,10.5,19.0,19.5,9.5,8.5,9.5,8.5,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,9.833333


In [61]:
y_train = train.loc[4320:, 'Y18']
Y_train = Y_train['Y18']
target = pd.concat([Y_train, y_train])

In [35]:
X_train = train.loc[:, 'X00':'X39']
X_train.head()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,...,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39
id,,,,,,,,,,,,,,,,,,,,,
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,...,69.1,8.2,10.7,1010.1,0.0,256.4,0.0,77.2,62.6,0.0
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,...,70.3,8.3,10.3,1010.1,0.0,215.4,0.0,77.3,63.5,0.0
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,...,71.5,8.0,9.7,1010.0,0.0,235.2,0.0,77.3,63.9,0.0
3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,...,73.2,7.7,9.4,1010.1,0.0,214.0,0.0,77.5,64.5,0.0
4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,...,74.3,7.4,9.4,1010.1,0.0,174.9,0.0,78.0,65.0,0.0


In [70]:
target = pd.DataFrame(target)
dataset= X_train.join(target, how='outer') # concat, append에서는 데이터 손실 발생
dataset

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,...,X31,X32,X33,X34,X35,X36,X37,X38,X39,Y18
id,,,,,,,,,,,,,,,,,,,,,
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,...,8.2,10.7,1010.1,0.00,256.4,0.0,77.2,62.6,0.0,10.666667
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,...,8.3,10.3,1010.1,0.00,215.4,0.0,77.3,63.5,0.0,10.444444
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,...,8.0,9.7,1010.0,0.00,235.2,0.0,77.3,63.9,0.0,10.166667
3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,...,7.7,9.4,1010.1,0.00,214.0,0.0,77.5,64.5,0.0,10.000000
4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,...,7.4,9.4,1010.1,0.00,174.9,0.0,78.0,65.0,0.0,9.833333
5,9.1,988.9,1.3,0.7,0.0,1009.1,989.6,12.0,1010.0,1010.0,...,7.3,9.1,1010.1,0.00,243.1,0.0,78.7,65.1,0.0,9.750000
6,9.1,988.8,2.4,1.9,0.0,1009.1,989.6,12.0,1010.1,1010.0,...,7.1,8.6,1010.1,0.00,195.8,0.0,78.6,65.5,0.0,9.444444
7,8.9,988.9,1.0,1.2,0.0,1009.2,989.7,11.9,1010.1,1010.1,...,7.1,8.8,1010.1,0.00,262.7,0.0,78.6,65.7,0.0,9.416667
8,9.0,988.9,1.5,0.1,0.0,1009.2,989.6,11.9,1010.0,1010.1,...,7.0,8.8,1010.1,0.00,236.1,0.0,78.8,66.5,0.0,9.305556


In [78]:
# 함수 적용 시 X00~X39+Y18 형식 표 만들기


def split_xy5(dataset, time_steps, y_column): 
    x, y = list(), list()
    for i in range(len(dataset)):
        x_end_number = i + time_steps # x의 끝 값(몇 개씩 자를 것인가)
        y_end_number = x_end_number + y_column # y의 시작값
        
        if y_end_number > len(dataset):
            break
        tmp_x = dataset[i:x_end_number, :] # x의 위치 => 행 지정, 열 전부
        tmp_y = dataset[x_end_number:y_end_number, 40] # y의 위치, 숫자는 '종가'(target)의 idx
        x.append(tmp_x)
        y.append(tmp_y)
        
    return np.array(x), np.array(y)

time_steps = 5
x, y =split_xy5(dataset, 5, 1) # Type error 발생
# print(x.shape)
# print(y.shape)
# print(x[0,:], '\n', y[0])

TypeError: '(slice(0, 5, None), slice(None, None, None))' is an invalid key

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_target, y_test = train_test_split(X_train, target, test_size=0.2)

In [21]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1], 1)

print(x_train.shape)
print(x_test.shape)

(3801, 40, 1)
(951, 40, 1)


In [23]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, BatchNormalization, Dropout

input1 = Input(shape=(40, 1))
dense1 = LSTM(256, activation='relu', return_sequences=True)(input1) 
dense1 = LSTM(256, activation='relu', return_sequences=False)(dense1) 
dense1 = Dense(128)(dense1) 
output1 = Dense(1)(dense1)

model = Model(inputs=input1, outputs=output1)

# 5. 모델 훈련시키기 
model.compile(loss='mse', 
              optimizer='adam',
              metrics=['mse'])
model.fit(x_train, y_train, epochs=50, batch_size=10, validation_split=0.1)

# 6. 평가 검증하기
loss, mse = model.evaluate()
print('loss :', loss)

ValueError: Input arrays should have the same number of samples as target arrays. Found 3801 input samples and 432 target samples.

In [66]:
from sklearn.metrics import mean_squared_error
model.fit(x_train, y_target)
y_pred = model.predict(x_test)

loss = mean_squared_error(y_test, y_pred) 
print("MSE : %.3f"% loss)

from sklearn.metrics import mean_squared_error

def RMSE(y_test, y_predict): # 실제 정답값, 모델을 통한 예측값 
    return np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE :', RMSE(y_test, y_pred))

MSE : 0.575
RMSE : 0.7584319848477917


In [67]:
test_x = test.loc[:, 'X00':'X39']

# 예측하기 
finetune_pred = model.predict(test_x)

In [68]:
# 제출 파일 만들기
submission = pd.DataFrame(data=finetune_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission(11).csv', index=True)

In [17]:
# loss(r2) 값 최대로 낮추고 submission 올려보기
# 목표 : Randomforestregressor보다 점수 높이기(20200314 달성)
# 목표 2: 50위 안에 들기 or 5점 대 달성

# loss 계산 지표 확인해볼 것 

# STANDARD: XGBRegressor(n_estimators=30, learning_rate=0.15, num_iteration=1000, max_depth=5)
# submission(9): n_estimators=40, learning_rate=0.15, num_iteration=1000, max_depth=5
# submission(10): n_estimators=40, learning_rate=0.12, num_iteration=1000, max_depth=5(점수 낮아짐)
# submission(11): n_estimators=42, learning_rate=0.13, num_iteration=1000, max_depth=5(점수 높아짐)